# This is in github
### Databricks PySpark Notebook: Simple Medallion Architecture + Unity Catalog

### Updated to use Unity Catalog directly (since Hive Metastore is disabled).
_This simplified notebook shows the basics of:_
- Bronze → Silver → Gold layers (medallion architecture)
- explode + aggregation
- Read/write to Catalog schema
- Performance tips (broadcast join, partitioning, Z-ORDER, query comparison)
- Short interview notes as comments

In [0]:
from pyspark.sql import functions as F, types as T

# -------------------------------------------------------------------
# SETUP (Unity Catalog)
# -------------------------------------------------------------------
CATALOG = "diggibyte"  # catalog name
SCHEMA = "poc"

# Create the catalog if it does not exist
spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}")
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE {SCHEMA}")


# -------------------------------------------------------------------
# BRONZE: Raw ingest using simplified JSON
# -------------------------------------------------------------------
bronze_data = [
    (
        1001,
        "AR-001",
        "2025-08-01T10:03:22Z",
        '[{"ar":"SAR-1","qty":2,"price":19.99},'
        ' {"ar":"SAR-2","qty":1,"price":5.25}]',
        "WELCOME10;FREESHIP",
        "Bengaluru",
        "IN",
    ),
    (
        1002,
        "AR-002",
        "2025-08-01T11:15:05Z",
        '[{"ar":"SAR-2","qty":4,"price":5.00},'
        ' {"ar":"SAR-3","qty":1,"price":99.00}]',
        "LOYAL5",
        "Mumbai",
        "IN",
    ),
    (
        1003,
        "AR-001",
        "2025-08-02T09:01:11Z",
        '[{"ar":"SAR-1","qty":1,"price":19.99},'
        ' {"ar":"SAR-4","qty":6,"price":2.00}]',
        "",
        "Delhi",
        "IN",
    ),
]

schema = "order_id INT, customer_id STRING, order_ts STRING, items_json STRING, promotions STRING, shipping_city STRING, shipping_country STRING"
bronze_df = spark.createDataFrame(bronze_data, schema)
bronze_tbl = f"{CATALOG}.{SCHEMA}.bronze_orders"
bronze_df.write.format("delta").mode("overwrite").saveAsTable(bronze_tbl)

# -------------------------------------------------------------------
# SILVER: Parse + explode items and promotions
# -------------------------------------------------------------------
item_schema = T.ArrayType(
    T.StructType(
        [
            T.StructField("ar", T.StringType()),
            T.StructField("qty", T.IntegerType()),
            T.StructField("price", T.DoubleType()),
        ]
    )
)

b = spark.table(bronze_tbl)
silver_df = (
    b.withColumn("order_ts", F.to_timestamp("order_ts"))
    .withColumn("items", F.from_json("items_json", item_schema))
    .withColumn("item", F.explode("items"))
    # Fix: keep non-promoted items with NULL promotion instead of dropping them
    .withColumn(
        "promotions_arr",
        F.when(F.length("promotions") > 0, F.split("promotions", ";")).otherwise(
            F.array(F.lit(None).cast("string"))
        ),
    )
    .select(
        "order_id",
        "customer_id",
        "order_ts",
        "shipping_city",
        "shipping_country",
        F.col("item.ar").alias("ar"),
        F.col("item.qty").alias("qty"),
        F.col("item.price").alias("price"),
        (F.col("item.qty") * F.col("item.price")).alias("item_total"),
        F.explode_outer("promotions_arr").alias("promotion"),
    )
)

silver_tbl = f"{CATALOG}.{SCHEMA}.silver_order_items"
(
    silver_df.write.format("delta")
    .mode("overwrite")
    .partitionBy("shipping_country")  # Partition by country for scale
    .saveAsTable(silver_tbl)
)

# -------------------------------------------------------------------
# GOLD: Aggregations
# -------------------------------------------------------------------
# Customer-level revenue
customer_gold_df = silver_df.groupBy("customer_id").agg(
    F.sum("item_total").alias("total_spent")
)

customer_gold_tbl = f"{CATALOG}.{SCHEMA}.gold_customer_revenue"
(
    customer_gold_df.write.format("delta")
    .mode("overwrite")
    .saveAsTable(customer_gold_tbl)
)
spark.sql(f"OPTIMIZE {customer_gold_tbl} ZORDER BY (customer_id)")

# Promotion-level revenue
promotion_gold_df = (
    silver_df.groupBy("promotion")
    .agg(F.sum("item_total").alias("total_spent"))
    .filter(F.col("promotion").isNotNull())
)

promotion_gold_tbl = f"{CATALOG}.{SCHEMA}.gold_promotion_revenue"
(
    promotion_gold_df.write.format("delta")
    .mode("overwrite")
    .saveAsTable(promotion_gold_tbl)
)
spark.sql(f"OPTIMIZE {promotion_gold_tbl} ZORDER BY (promotion)")

# City-level revenue
city_gold_df = silver_df.groupBy("shipping_city").agg(
    F.sum("item_total").alias("total_spent")
)

city_gold_tbl = f"{CATALOG}.{SCHEMA}.gold_city_revenue"
(city_gold_df.write.format("delta").mode("overwrite").saveAsTable(city_gold_tbl))
spark.sql(f"OPTIMIZE {city_gold_tbl} ZORDER BY (shipping_city)")

# Daily revenue (time-based aggregation)
daily_gold_df = silver_df.groupBy(F.to_date("order_ts").alias("order_date")).agg(
    F.sum("item_total").alias("daily_revenue")
)

daily_gold_tbl = f"{CATALOG}.{SCHEMA}.gold_daily_revenue"
(daily_gold_df.write.format("delta").mode("overwrite").saveAsTable(daily_gold_tbl))
spark.sql(f"OPTIMIZE {daily_gold_tbl} ZORDER BY (order_date)")

# -------------------------------------------------------------------
# PERFORMANCE TIP (broadcast join)
# -------------------------------------------------------------------
small_dim = spark.createDataFrame(
    [
        ("SAR-1", "Clothes"),
        ("SAR-2", "Shoes"),
        ("SAR-3", "Electronics"),
        ("SAR-4", "Snacks"),
    ],
    ["ar", "category"],
)
joined = silver_df.join(F.broadcast(small_dim), "ar", "left")
joined.show()

# -------------------------------------------------------------------
# PERFORMANCE COMPARISON: With vs Without Z-ORDER
# -------------------------------------------------------------------
import time

# Without Z-ORDER (raw filter scan)
start = time.time()
spark.sql(f"SELECT * FROM {customer_gold_tbl} WHERE customer_id = 'AR-001'").collect()
print("Query time without Z-ORDER:", time.time() - start, "seconds")

# With Z-ORDER (optimized table)
start = time.time()
spark.sql(f"SELECT * FROM {customer_gold_tbl} WHERE customer_id = 'AR-001'").collect()
print("Query time with Z-ORDER:", time.time() - start, "seconds")

In [0]:
# Sql to list table contents
%sql
Select * from diggibyte.poc.bronze_orders;
select * from diggibyte.poc.gold_city_revenue;
select * from diggibyte.poc.gold_customer_revenue;
select * from diggibyte.poc.gold_daily_revenue;
select * from diggibyte.poc.gold_promotion_revenue;
select * from diggibyte.poc.silver_order_items;

In [0]:
# DataFrame API to list the table records
df = spark.table("diggibyte.poc.bronze_orders")
df.show()

df = spark.table("diggibyte.poc.gold_city_revenue")
df.show()

df = spark.table("diggibyte.poc.gold_customer_revenue")
df.show()

df = spark.table("diggibyte.poc.gold_daily_revenue")
df.show()

df = spark.table("diggibyte.poc.gold_promotion_revenue")
df.show()

df = spark.table("diggibyte.poc.silver_order_items")
df.show()
 